In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord as coord
import astropy.units as u
from astropy.coordinates import Angle
from astropy.time import Time
import warnings
warnings.filterwarnings('ignore')

###  Step 1:  Importing gamma ray burst data

<font color = 'purple'>Data I have used for gamma ray burst was taken form this site below.</font>

https://swift.gsfc.nasa.gov/results/batgrbcat/summary_cflux/summary_general_info/summary_general.txt


In [2]:
column_names = ['GRBname', 'RA_ground', 'DEC_ground']
df_grb = pd.read_csv('/home/astroguy/frb_summerresearch/grb_data_copy.csv', delimiter='|', header = 0)
df_grb.head(1)

,GRBname,Trig_ID,Trig_time_met,Trig_time_UTC,RA_ground,DEC_ground,Image_position_err,Image_SNR,T90,T90_err,T50,T50_err,Evt_start_sincetrig,Evt_stop_sincetrig,pcode,Trigger_method,XRT_detection,"comment,,,"
0,GRB200829A,993768,620402400.000,2020-08-29T13:59:34.457300,2.511393e+02,7.236396e+01,1.821205e+00,1.291746e+01,1.310000e+01,3.079558e+00,2.684000e+00,4.882629e-02,-239,963,0.8828,image trigger,Yes,",,,"


<font color = 'purple'>Note:  According to the description in the catalog RA_ground and DEC_ground both has been measured in the unit of degree.</font> 

In [3]:
# Taking only necessary subset of the whold data from the catalog

df_grb_subset = df_grb.iloc[:,[0,3,4,5]]
print(df_grb_subset.shape)
df_grb_subset.head(1)

(1391, 4)


,GRBname,Trig_time_UTC,RA_ground,DEC_ground
0,GRB200829A,2020-08-29T13:59:34.457300,2.511393e+02,7.236396e+01


In [4]:
# Since some data are missing we want to get rid of those rows. Using for loop to pick the rows with no data.

empty_row_num = []

for num in np.arange(0, len(df_grb_subset.iloc[:])):
    if df_grb_subset.iloc[num, 1] == ' N/A ' or df_grb_subset.iloc[num, 2] == ' N/A ':
        empty_row_num.append(num)

empty_row_num

[222, 271, 325, 430, 483, 786, 1110, 1188, 1283, 1389]

In [5]:
# Using drop function to drop all the rows with no data. Ten rows with no data have been removed.

df_grb_subset.drop(empty_row_num, axis = 0, inplace = True)
print(df_grb_subset.shape)

(1381, 4)


###  Step 2: Working with fast radio burst data

<font color = 'purple'>Data taken from link below</font>

https://www.herta-experiment.org/frbstats/catalogue

In [6]:
columns = ['ra', 'utc', 'dec', 'dm', 'flux', 'redshift', 'frb', 'l', 'b', 'width', 'fluence']
df_herta = pd.read_csv('herta_frb_catalogue.csv', usecols=columns).replace('-', '0')
df_herta.head(1)

,frb,utc,ra,dec,l,b,dm,flux,width,fluence,redshift
0,FRB 20010125A,2001-01-25 0:29:16,19:06:53,-40:37:14,356.64,-20.02,790.3,0.54,0,0,0.7083


<font color = 'purple'>Unit of RA in the catalog is hour:min:sec and that of declination is in deg:arcmin:arcsec</font>

In [7]:
herta_subset = df_herta.iloc[:, [0, 1,2,3]]
herta_subset.tail(1)

,frb,utc,ra,dec
805,FRB 20220501C,2022-05-01 2:11:09,23:29:50,-32:27:00


<font color = 'purple'>We convert RA and DEC to the unit of degree to make it uniform with the unit of RA and DEC in GRB data. We use astropy library to convert coordinates to degress. </font>

Reference:  https://docs.astropy.org/en/stable/coordinates/angles.html

<font color = 'purple'>This site maintained by penstate can be used as checker to see if our conversion has been done correctly.</font>

https://www.swift.psu.edu/secure/toop/convert.htm

In [8]:
RA_in_degrees  = []
DEC_in_degrees = []

for k in np.arange(0, len(herta_subset['frb'])):
    RA_degree = Angle(herta_subset['ra'][k], unit = 'hourangle')   # RA is in  hr:min:sec
    DEC_degree = Angle(herta_subset['dec'][k], unit = 'degree')    # DEC is in deg:arcmin:arcsec
    RA_in_degrees.append(RA_degree.degree)
    DEC_in_degrees.append(DEC_degree.degree)
    
herta_subset['RA_in_deg'] = RA_in_degrees
herta_subset['DEC_in_deg'] = DEC_in_degrees

herta_subset.head(1)

,frb,utc,ra,dec,RA_in_deg,DEC_in_deg
0,FRB 20010125A,2001-01-25 0:29:16,19:06:53,-40:37:14,286.720833,-40.620556


In [9]:
# This function calculates distance using formula shown below.

def distance(ra_frb, ra_grb, dec_frb, dec_grb):
    diff_ra = ra_frb - ra_grb
    diff_dec = dec_frb - dec_grb
    avg_dec = (dec_frb + dec_grb)/2
    frb_grb_dist = np.sqrt((diff_ra*np.cos(avg_dec*(np.pi/180)))**2 + diff_dec**2)
    return frb_grb_dist

$$ dist = \sqrt{ ({\Delta\alpha\cos{\delta}})^2 + {\Delta{\delta}}^2} $$

where, 

<br>$ \Delta \alpha = ra_{1} - ra_{2} $
<br>$\Delta \delta = de_{1} - de_{2} $ 
<br>$ \delta = \frac{de_{1} + de_{2}}{2}$

In [10]:
dist_s_no = []  # This list will contain serial number as first value, grb index number as second value and 
                # distance calculated as the third value in the list.
    
for m in np.arange(0, len(herta_subset['frb'])):
    frb_ra = herta_subset['RA_in_deg'][m]
    frb_dec = herta_subset['DEC_in_deg'][m]

    dist_list = []
    for n in np.arange(0, len(df_grb_subset.iloc[:])):
        grb_ra = float(df_grb_subset.iloc[n][2])
        grb_dec = float(df_grb_subset.iloc[n][3])
        
        dist = distance(frb_ra, grb_ra, frb_dec, grb_dec)
        dist_list.append(dist)
        
    min_dist_val = min(dist_list)
    grb_index = dist_list.index(min_dist_val)
    dist_s_no.append([m, grb_index, min_dist_val])
    
dist_s_no[0:10]  # Just a glimps of distance calculated between frb and grb source

[[0, 760, 2.596752510596917],
 [1, 291, 2.48303805797005],
 [2, 167, 3.1081312643501358],
 [3, 1362, 0.9603655074490652],
 [4, 1030, 0.932885260827942],
 [5, 60, 2.83975721994394],
 [6, 928, 1.637962389137355],
 [7, 1235, 2.036364428218165],
 [8, 934, 2.3771608506085227],
 [9, 787, 4.587342672335706]]

In [11]:
dist_less_than_one_group = []  # List of frb/grb pairs at distance less than 1 unit.
grb_serial_no = []              # List of grb serial numbers.

for num1 in np.arange(0, len(dist_s_no)):
    grb_serial_no.append(dist_s_no[num1][1])
    if 0<dist_s_no[num1][2]<1:
        dist_less_than_one_group.append(dist_s_no[num1])

print('There are', len(dist_less_than_one_group), 'pair of frb/grb source at distance less than 1 unit')

There are 89 pair of frb/grb source at distance less than 1 unit


In [12]:
# In this part we are trying to see the frequency of repetition of grb that has distance less than 1 unit with frb.

grb_code_no = []
grb_freq_val = []

for num2 in np.arange(0, len(grb_serial_no)):
    serial_no = grb_serial_no[num2]
    freq = grb_serial_no.count(serial_no)
    if grb_serial_no[num2] not in grb_code_no:
        grb_code_no.append(serial_no)
        grb_freq_val.append(freq)
        
        
# Now we pair up grb reference serial no and its frequency of pairing up with frb sources. First value in the list
# refers to the grb index number in the data list and second number indicates the frequency of repetition.

grb_code_freq = []
for num3 in np.arange(0, len(grb_code_no)):
    grb_code_freq.append([grb_code_no[num3], grb_freq_val[num3]])

grb_code_freq[0:10]

[[760, 1],
 [291, 1],
 [167, 1],
 [1362, 1],
 [1030, 1],
 [60, 1],
 [928, 1],
 [1235, 2],
 [934, 1],
 [787, 1]]

### Matching FRB data with GRB data where both events occured with an interval of 1 day.

In [13]:
herta_subset.tail(1)

,frb,utc,ra,dec,RA_in_deg,DEC_in_deg
805,FRB 20220501C,2022-05-01 2:11:09,23:29:50,-32:27:00,352.458333,-32.45


You can use this site from NASA as a confirmation check if the date has been converted correctly to Julian date.

https://ssd.jpl.nasa.gov/tools/jdc/#/cd

In [14]:
grb_jd = Time(pd.to_datetime(df_grb_subset.iloc[:,1])).jd  # Triger time of GRB in julian day
frb_jd = Time(pd.to_datetime(herta_subset['utc'])).jd      # Time of FRB converted to julian day

print(len(grb_jd))
print(len(frb_jd))

1381
806


In [25]:
frb_grb_day_matching = []  # This is an empty list created to store those frb and grb pair number which occured 
                           # within an interval of 1 day. First number indexes FRB and second indicates GRB.
                           # Third number in the group is the interval measured in day.
        
for n_frb in np.arange(0, len(frb_jd)):
    for n_grb in np.arange(0, len(grb_jd)):
        if abs(frb_jd[n_frb] - grb_jd[n_grb]) <= 1:
            frb_grb_day_matching.append([n_frb, n_grb, abs(frb_jd[n_frb] - grb_jd[n_grb])])


len(frb_grb_day_matching)  # Sample from the list created where first number is frb index, second grb index and third
                          # number is the time interval between the frb and grb event.

328

### Matching of those frb and grb data that occured less than 1 day apart and lie at a position less than 1 unit of distance in the sky.

Now I write a prog that will take frb and grb index from frg_grb_day_matching and calculate the distance between 
them using the distance function I have already written.


In [36]:
time_and_position_matched_list = []

for ind in np.arange(0, len(frb_grb_day_matching)):
    
            
    frb_row_index = frb_grb_day_matching[ind][0]
#     print('FRB index', frb_row_index)
    grb_row_index = frb_grb_day_matching[ind][1]
#     print('GRB index', grb_row_index)

    ra_frb = herta_subset['RA_in_deg'][frb_row_index]
#     print('RA for FRB', ra_frb)
    dec_frb = herta_subset['DEC_in_deg'][frb_row_index]
#     print('DEC for FRB', dec_frb)

    ra_grb = float(df_grb_subset.iloc[grb_row_index, 2])
#     print('RA for GRB', ra_grb)
    dec_grb = float(df_grb_subset.iloc[grb_row_index, 3])
#     print('DEC for GRB', dec_grb)

    frb_grb_dist = distance(ra_frb, ra_grb, dec_frb, dec_grb)
#     print('Distance between frb and grb sources', frb_grb_dist)
    
    if frb_grb_dist<= 7:
        time_and_position_matched_list.append([frb_row_index, grb_row_index])

In [37]:
time_and_position_matched_list

[[329, 101]]

Results:

One FRB & GRB event occured within an interval of 2 days and 2 unit distant from each other. These two FRB and
GRB event are:

<font color = red>FRB 20190131C </font>     &&  <font color = red> GRB190202A </font>



Number of FRB and GRB pairs that occurred one day apart and are at less than 1 unit of distance = 0

Number of FRB and GRB pairs that occurred one day apart and are at less than 5 unit of distance = 0

Number of FRB and GRB pairs that occurred one day apart and are at less than 7 unit of distance = 1

These FRB and GRB event are:

<font color = red> FRB 20190124A </font>   &&   <font color = red>GRB190123A</font>
